# Recalculations, Plots, and Analysis of Data

In [ ]:
from numpy import *
import astropy.units as u
from astropy.constants import G, M_jup, R_jup, M_earth, R_earth, L_sun, M_sun, R_sun
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import extinction as ex
import scipy.stats as st

In [ ]:
#import scaling relation coefficients
rel = pd.read_csv('scalingrels_a17.csv')
rel.set_index('Tracer', inplace=True)

## Functions:
1. Flux to luminosity

2. Luminosity to accretion luminosity

3. Accretion luminosity to accretion rate

4. Get accretion rate starting from line flux

In [ ]:
def line_lum(line_flux, dist):
    """
    Calculate line luminosity given line flux and distance
    assuming line flux is extinction corrected.
    """
    
    line_lum = 4 * pi * (dist*u.pc)**2 * line_flux * u.erg / (u.s * (u.cm)**2)
    line_lum = line_lum.decompose().to(u.W)
    return line_lum/u.W

def accr_lum(L_line, tracer, L_line_err = 0*u.W):
    """
    Translate a line luminosity to accretion luminosity using empirical
    relationships from Alcala et al. 2017.
    
    Included tracers are:
    'Ha'
    'PaB'
    'BrG'
    """
        
    a, a_err, b, b_err = rel['a'][tracer],rel['a_err'][tracer],rel['b'][tracer],rel['b_err'][tracer]
    
    log_L_acc = b + a * log10(L_line*u.W/L_sun)
    
    L_acc = 10**log_L_acc*L_sun/u.W
    
    #error propagation
    
    #c_err = (L_line_err)/(log(10) * L_line)
    #ac_err = a * log10(L_line/L_sun) * ((a_err/a)**2 + (c_err/log10(L_line/L_sun))**2)**0.5
    #log_L_acc_err = (b_err**2 + ac_err**2)**0.5
    #L_acc_err = L_acc * log(10) * log_L_acc_err

    return L_acc

def acc_rate(L_acc, R, M):
    """
    Translate an accretion luminosity and planet mass/radius
    to accretion rate in Solar masses per year.
    """
    mdot = 1.25*L_acc*u.W*R*u.R_sun/(G*M*u.M_sun)
    mdot = mdot.decompose().to(u.M_sun/u.yr)
    return(mdot/(u.M_sun/u.yr))

def get_rate(line_flux, d, t, R, M):
    """
    Turn a line flux into an accretion rate, 
    given the distance, tracer, object radius, and object mass.
    
    Line flux should be in erg/s/cm^2; distance in pc;
    tracer either Ha, PaB, or BrG; radius in solar radii,
    and mass in solar masses.
    """
    l_line = line_lum(line_flux,d)
    l_acc = accr_lum(l_line, t)
    mdot = acc_rate(l_acc, R, M)
    
    return mdot

## Uniform scaling relations and new distances

In [ ]:
#read in CSV and make list of tracers
db = pd.read_csv('recalculations.csv')
tracers = ['Ha','PaB','BrG']

Recalculation:

In [ ]:
#get accretion rate for each tracer for each object

for i in db.index:
    #get relevant parameters: d, R, M
    
        #use new distance if it's available; else, use existing distance
    d = db.loc[i,'Plx Distance']
    if isnan(d):
        d = db.loc[i,'Object Distance']
    R = db.loc[i,'Object Radius']
    M = db.loc[i,'Object Mass, Original']
    #ignore point if missing M or R
    if (isnan(M) or isnan(R)):
        continue
        
    for t in tracers:
        lf = db.loc[i,t + ' Line Flux']
        
        if isnan(lf)==False:
            
            #extinction corrections: this is not automatic—need to know which need correction
            if db.loc[i,'Reference']=='Natta 2006':
                lf = ex.remove(db.loc[i,'A_J'],lf)
            if db.loc[i,'Reference']=='Kalari 2015':
                lf = ex.remove(db.loc[i,'A_V'],lf)
            
            #get the accretion rate
            db.at[i,t + ' Accr Rate'] = get_rate(lf, d, t, R, M)

In [ ]:
#take average accretion rate across the three tracers
db['Avg Accr Rate'] = db[[t + ' Accr Rate' for t in tracers]].mean(axis=1)

Plotting and analysis:

In [ ]:
#Take relevant columns and drop NaNs
new_est = db[['Source','Reference','Object Mass, Original', 'Avg Accr Rate','Old Accr Rate']].dropna()

#lists of mass (m), old accr rate (m dot, or md), and new accr rate
m = log10(new_est['Object Mass, Original'].tolist())
md_old = log10(new_est['Old Accr Rate'].tolist())
md_new = log10(new_est['Avg Accr Rate'].tolist())

In [ ]:
#best-fit parameters
sl_old, incpt_old = polyfit(m,md_old,1)
sl_new, incpt_new = polyfit(m,md_new,1)
print('Old Best-Fit: Mdot =', sl_old,'M', incpt_old)
print('New Best-Fit: Mdot =', sl_new,'M', incpt_new)

In [ ]:
#correlations
r_old = st.pearsonr(m,md_old)
r_new = st.pearsonr(m,md_new)
print('Old r:', r_old)
print('New r:', r_new)

In [ ]:
print('Number of rows included:', len(new_est))

Plot 1: Compare old and new

In [ ]:
#plotting:
plt.figure(figsize=(7,7/1.625),dpi=750)

plt.scatter(m,md_old,color='k',alpha=0.1,label='r='+str(round(r_old[0],2)),
           edgecolors='none',s=70)
plt.scatter(m,md_new,color='b',alpha=0.25,label='r='+str(round(r_new[0],2)),
           edgecolors='none',s=70)

x = arange(-2.5,1,0.5)
plt.xlim(-2.5,0.5); plt.ylim(-14,-6)

plt.plot(x, sl_old*x+incpt_old, color='k', alpha=0.5,label='y='+str(round(sl_old,2))+'x'+str(round(incpt_old,2)))
plt.plot(x, sl_new*x+incpt_new, color='b', label='y='+str(round(sl_new,2))+'x'+str(round(incpt_new,2)))

plt.legend(frameon=False); plt.title('Accretion Rate vs Mass')

plt.xlabel('log $M$  $(M_{\odot})$'); plt.ylabel('log $\dot M$  $(M_{\odot}/yr)$')

Plot 2: Only new

In [ ]:
#plotting:
plt.figure(figsize=(7,7/1.625),dpi=750)

plt.scatter(m,md_new,color='b',alpha=0.2,label='r='+str(round(r_new[0],2)),
           edgecolors='none',s=70)

x = arange(-2.5,1,0.5)
plt.xlim(-2.5,0.5); plt.ylim(-14,-6)

plt.plot(x, sl_new*x+incpt_new, color='k', label='y='+str(round(sl_new,2))+'x'+str(round(incpt_new,2)))

plt.legend(frameon=False); plt.title('Accretion Rate vs Mass')

plt.xlabel('log $M$  $(M_{\odot})$'); plt.ylabel('log $\dot M$  $(M_{\odot}/yr)$')

## Uniform scaling relations, new distances, new masses

In [ ]:
#read in CSV again
db2 = pd.read_csv('recalculations.csv')

Recalculation:

In [ ]:
#get accretion rates
#mass_used: one list of the masses used to recalculate accretion rate
mass_used = []

for i in db2.index:  
    #use new distance, mass, and radius if it's available; else, use existing value
    d = db2.loc[i,'Plx Distance']
    if isnan(d):
        d = db2.loc[i,'Object Distance']
    R = db2.loc[i,'Object Radius']
    M = db2.loc[i,'New Mass']
    if isnan(M):
        M = db2.loc[i,'Object Mass, Original']
        
    mass_used.append(M)

    #ignore if missing M or R
    if (isnan(M) or isnan(R)):
        continue
            
    for t in tracers:
        lf = db2.loc[i,t + ' Line Flux']
        if isnan(lf):
            continue   
        #extinction corrections when necessary
        if db2.loc[i,'Reference']=='Natta 2006':
            lf = ex.remove(db2.loc[i,'A_J'],lf) 
        if db2.loc[i,'Reference']=='Kalari 2015':
            lf = ex.remove(db2.loc[i,'A_V'],lf)
            
        #get the accretion rate
        db2.at[i,t + ' Accr Rate'] = get_rate(lf, d, t, R, M)

In [ ]:
#add mass used as a database column. This enables pulling all the masses as one list.
db2['Adopted Mass'] = mass_used

In [ ]:
#Take average accretion rate across the three tracers
db2['Avg Accr Rate'] = db2[[t + ' Accr Rate' for t in tracers]].mean(axis=1)

Plotting and analysis:

In [ ]:
#Take relevant columns and drop NaNs
new_est2 = db2[['Source','Reference','Object Mass, Original','Adopted Mass',
           'Avg Accr Rate','Old Accr Rate']].dropna()

In [ ]:
#lists of mass, old accr rate, and new accr rate
m_old = log10(new_est2['Object Mass, Original'].tolist())
m_new = log10(new_est2['Adopted Mass'].tolist())
md_old2 = log10(new_est2['Old Accr Rate'].tolist())
md_new2 = log10(new_est2['Avg Accr Rate'].tolist())

In [ ]:
#best fit parameters
sl_old2, incpt_old2 = polyfit(m_old,md_old2,1)
sl_new2, incpt_new2 = polyfit(m_new,md_new2,1)
print('Old Best-Fit: y = ', sl_old2,'x ', incpt_old2)
print('New Best-Fit: y = ', sl_new2,'x ', incpt_new2)

In [ ]:
#correlations
r_old2 = st.pearsonr(m_old,md_old2)
r_new2 = st.pearsonr(m_new,md_new2)
print('Old r:', r_old2)
print('New r:', r_new2)

In [ ]:
print('Number of rows included:', len(new_est2))

Plot 1: compare old and new

In [ ]:
plt.figure(figsize=(7,7/1.625),dpi=400)

plt.scatter(m_old,md_old2,color='k',alpha=0.1,label='r='+str(round(r_old2[0],2)),
           edgecolors='none',s=70)
plt.scatter(m_new,md_new2,color='b',alpha=0.25,label='r='+str(round(r_new2[0],2)),
           edgecolors='none',s=70)

x = arange(-2.5,1,0.5)
plt.xlim(-2.5,0.5); plt.ylim(-14,-6)

plt.plot(x, sl_old2*x+incpt_old2, color='k', alpha=0.5,label='y='+str(round(sl_old2,2))+'x'+str(round(incpt_old2,2)))
plt.plot(x, sl_new2*x+incpt_new2, color='b', label='y='+str(round(sl_new2,2))+'x'+str(round(incpt_new2,2)))

plt.legend(frameon=False); plt.title('Accretion Rate vs Mass')

plt.xlabel('log $M$  $(M_{\odot})$'); plt.ylabel('log $\dot M$  $(M_{\odot}/yr)$')

Plot 2: only new

In [ ]:
plt.figure(figsize=(7,7/1.625),dpi=400)

plt.scatter(m_new,md_new2,color='b',alpha=0.25,label='r='+str(round(r_new2[0],2)),
            edgecolors='none',s=70)

x = arange(-2.5,1,0.5)
plt.xlim(-2.5,0.5); plt.ylim(-14,-6)

plt.plot(x, sl_new2*x+incpt_new2, color='k', label='y='+str(round(sl_new2,2))+'x'+str(round(incpt_new2,2)))

plt.legend(frameon=False); plt.title('Accretion Rate vs Mass')

plt.xlabel('log $M$  $(M_{\odot})$'); plt.ylabel('log $\dot M$  $(M_{\odot}/yr)$')

Plot 3: "Money Plot": new plot with limits and text

In [ ]:
plt.figure(figsize=(7,7/1.625),dpi=400)

plt.scatter(m_new,md_new2,color='b',alpha=0.25,label='r='+str(round(r_new2[0],2)),
            edgecolors='none',s=70)

x = arange(-2.5,1,0.5)
plt.xlim(-2.5,0.5); plt.ylim(-14,-6)

plt.plot(x, sl_new2*x+incpt_new2, color='k', label='y='+str(round(sl_new2,2))+'x'+str(round(incpt_new2,2)))

plt.legend(frameon=False); plt.title('Accretion Rate vs Mass')
plt.xlabel('log $M$  $(M_{\odot})$'); plt.ylabel('log $\dot M$  $(M_{\odot}/yr)$')

#limits:
plt.plot([log10(13/1048),log10(13/1048)],[-14,-6],color='k',linewidth=3)
plt.plot([log10(80/1048),log10(80/1048)],[-14,-6],color='k',linewidth=3)

#text:
plt.text(log10(13/1048)-.09,-10,'Deuterium burning limit',rotation='vertical',fontsize=9)
plt.text(log10(80/1048)-.09,-9.9,'Hydrogen burning limit',rotation='vertical',fontsize=9)
plt.text(-2.35,-6.5,'Planets',fontsize=10)
plt.text(-1.8,-6.5,'Brown Dwarfs',fontsize=10)
plt.text(-0.5,-6.5,'Stars',fontsize=10)

## Connect identical sources on "money plot"

In [ ]:
sourceSorted = new_est2.sort_values(by='Source')

In [ ]:
sourceSortedM = log10(sourceSorted['Adopted Mass'].tolist())
sourceSortedMdot = log10(sourceSorted['Avg Accr Rate'].tolist())

In [ ]:
#plotting: can use 

plt.figure(figsize=(7,7/1.625),dpi=400)

plt.scatter(sourceSortedM,sourceSortedMdot,
            color='b',alpha=0.15,label='r='+str(round(r_new2[0],2)),
            edgecolors='none',s=70)

plt.plot(x, sl_new2*x+incpt_new2, color='k', label='y='+str(round(sl_new2,2))+'x'+str(round(incpt_new2,2)))

#connect same sources with dotted lines
for i in range(len(sourceSorted)-1):
    if sourceSorted.loc[sourceSorted.index[i],'Source']==sourceSorted.loc[sourceSorted.index[i+1],'Source']:
        plt.plot([sourceSortedM[i],sourceSortedM[i+1]],[sourceSortedMdot[i],sourceSortedMdot[i+1]],
                 color='b',linestyle='-.',alpha=0.6)

plt.xlabel('log $M$  $(M_{\odot})$'); plt.ylabel('log $\dot M$  $(M_{\odot}/yr)$')
plt.xlim(-2.5,0.5); plt.ylim(-14,-6)
plt.legend(frameon=False,loc='lower right'); plt.title('Accretion Rate vs Mass') 

#limits:
plt.plot([log10(13/1048),log10(13/1048)],[-14,-6],color='k',linewidth=3)
plt.plot([log10(80/1048),log10(80/1048)],[-14,-6],color='k',linewidth=3)

#text:
plt.text(log10(13/1048)-.09,-10,'Deuterium burning limit',rotation='vertical',fontsize=9)
plt.text(log10(80/1048)-.09,-9.9,'Hydrogen burning limit',rotation='vertical',fontsize=9)
plt.text(-2.35,-6.5,'Planets',fontsize=10)
plt.text(-1.8,-6.5,'Brown Dwarfs',fontsize=10)
plt.text(-0.5,-6.5,'Stars',fontsize=10)